In [1]:
from typing import Optional, List
from glob import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import gc
import logging
import yaml
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr
from libs import wormup as wu

import warnings
warnings.filterwarnings("ignore")

from contextlib import contextmanager
from catalyst.dl import SupervisedRunner, State, CallbackOrder, Callback, CheckpointCallback

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
with open('../settings/catalyst_efficient_b0_base.yaml') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
    
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    
    
set_seed(1213)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.transform = tr.Compose([
            tr.OneOf([
                tr.GaussianNoiseSNR(min_snr=10),
                tr.PinkNoiseSNR(min_snr=10)
            ]),
            tr.PitchShift(max_steps=2, sr=SR),
            #tr.TimeStretch(),
            #tr.TimeShift(sr=sr),
            tr.VolumeControl(mode="sine")
        ])
                
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def get_random_duration(self, duration=10):
        start_sec = random.randint(0, 60-duration)
        end_sec = start_sec + 10
            
        return start_sec, end_sec
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration

        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start_sec = t_min - front_time
            end_sec = t_max + back_time
            
        return start_sec, end_sec

    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        start_sec, end_sec = self.get_duration(t_min, t_max, 10)
            
        record_path = self.path + recording_id + '.flac'
        y = self.load(record_path)
        y =  y[start_sec*SR:end_sec*SR]
        
        if self.train:
            y = self.transform(y)
                    
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return {"waveform": y.astype(np.float32), "targets": target}

In [6]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

In [7]:
class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [8]:
class EfficientNetSED(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        pretrained=False,
        num_classes=24,
        spectrogram_params={},
        logmel_extractor_params={},
        spec_augmenter_params={},
        pce_params={}
    ):
        super().__init__()
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        self.interpolate_ratio = 30  # Downsampled ratio
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if pretrained:
            self.base_model = EfficientNet.from_pretrained(base_model_name)
        else:
            self.base_model = EfficientNet.from_name(base_model_name)

        in_features = self.base_model._fc.in_features

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlockV2(in_features, num_classes, activation="sigmoid")

        self.init_weight()
        
    def mixup(self, x):
        sizws = x.size()
        #lam = torch.from_numpy(self.random_state.beta(self.mixup_alpha, self.mixup_alpha, (sizws[0], 1))).cuda()
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        #out = (x.view(sizws[0], -1) * lam + x[index].squeeze().view(sizws[0], -1) * (1-lam)).view(sizws[0], sizws[1], sizws[2], sizws[3],)
        out = (x * lam + x[index].squeeze() * (1-lam))
        return out.float(), {'lam': lam, 'index': index}

    def init_weight(self):
        init_layer(self.fc1)

    def forward(self, input):        
        x = self.spectrogram_extractor(input)
        x = self.logmel_extractor(x)
        
        x_mels = self.logmel_extractor.power_to_db(x)
        x_pcen = self.pcen_converter(x) 
        x_clear = self.logmel_extractor.power_to_db(x ** 1.5)
        
        
        x = torch.cat((x_mels,x_pcen,x_clear),1)
        #x = torch.cat((x,x,x),1)
        
        frames_num = x.size(2)
        
        if self.training:
            x, mix_info = self.mixup(x)
            x = self.spec_augmenter(x)
        else:
            mix_info = None
                
        # (batch_size, channels, freq, frames)
        x = self.base_model.extract_features(x)

        # (batch_size, channels, frames)
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "segmentwise_output": segmentwise_output,
            "logit": logit,
            "framewise_logit": framewise_logit,
            "clipwise_output": clipwise_output,
            "mixup": mix_info
        }

        return output_dict

In [9]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma

    def forward(self, logit, target):
        target = target.float()
        max_val = (-logit).clamp(min=0)
        loss = logit - logit * target + max_val + \
            ((-max_val).exp() + (-logit - max_val).exp()).log()

        invprobs = F.logsigmoid(-logit * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        if len(loss.size()) == 2:
            loss = loss.sum(dim=1)
        return loss.mean()

class ImprovedFocalLoss(nn.Module):
    def __init__(self, weights=[1, 1]):
        super().__init__()

        self.focal = FocalLoss()
        self.focal2 = FocalLoss()
        self.weights = weights

    def forward(self, input, target, mixup_info=None):
        input_ = input["logit"]
        target = target.float()

        framewise_output = input["framewise_logit"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        normal_loss = self.focal(input_, target)
        auxiliary_loss = self.focal(clipwise_output_with_max, target)
        
        loss1 = self.weights[0] * normal_loss + self.weights[1] * auxiliary_loss
        
        if mixup_info is None:
            return loss1
        else:
            normal_loss_mix = self.focal2(input_, target[mix_info['index']].squeeze())
            auxiliary_loss_mix = self.focal2(clipwise_output_with_max, target[mix_info['index']].squeeze())
            loss2 = self.weights[0] * normal_loss_mix + self.weights[1] * auxiliary_loss_mix
            
            return loss1 * mix_info['lam'] + loss2 * (1 - mix_info['lam'])

In [10]:
class LWLRAP(Callback):
    def __init__(self,
                 input_key: str = "targets",
                 output_key: str = "logits",
                 model_output_key: str = "framewise_output",
                 mixup_key: str = 'mixup',
                 prefix: str = "LWLRAP"):
        super().__init__(CallbackOrder.Metric)
        self.input_key = input_key
        self.output_key = output_key
        self.model_output_key = model_output_key
        self.mixup_key = mixup_key
        self.prefix = prefix
        
    def LWLRAP(self, preds, labels):
        # Ranks of the predictions
        ranked_classes = torch.argsort(preds, dim=-1, descending=True)
        # i, j corresponds to rank of prediction in row i
        class_ranks = torch.zeros_like(ranked_classes)
        for i in range(ranked_classes.size(0)):
            for j in range(ranked_classes.size(1)):
                class_ranks[i, ranked_classes[i][j]] = j + 1
        # Mask out to only use the ranks of relevant GT labels
        ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
        # All the GT ranks are in front now
        sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
        # Number of GT labels per instance
        num_labels = labels.sum(-1)
        pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
        score_matrix = pos_matrix / sorted_ground_truth_ranks
        score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
        scores = score_matrix * score_mask_matrix
        score = scores.sum() / labels.sum()
        return score.item()
    
    def mixup_socre(self, cor, x, y, mix_info):
        return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

    def on_batch_start(self, state: State):
        self.batch_score_train: List[np.ndarray] = []
        self.batch_score_val: List[np.ndarray] = []
        
    def on_batch_end(self, state: State):
        targ = state.input[self.input_key].detach().cpu()
        out = state.output[self.output_key]
        mixup_info = out[self.mixup_key]

        framewise_logit = out[self.model_output_key].detach().cpu().max(1)[0]

        if mixup_info is None:
            score = self.LWLRAP(framewise_logit, targ)
        else:
            score = self.mixup_socre(self.LWLRAP, framewise_logit, targ, mixup_info)
        
        state.batch_metrics[self.prefix] = score
        
        if state.is_valid_loader:
            self.batch_score_val.append(score)
        else:
            self.batch_score_train.append(score)

    def on_loader_end(self, state: State):
        if state.is_valid_loader:
            score = np.mean(self.batch_score_val)
            state.loader_metrics[self.prefix] = score
            state.epoch_metrics[state.valid_loader + "_epoch_" +
                                self.prefix] = score
        else:
            score = np.mean(self.batch_score_train)
            state.loader_metrics[self.prefix] = score
            state.epoch_metrics["train_epoch_" + self.prefix] = score

In [11]:
train_tp = pd.read_csv('../../data/train_tp.csv')
train_fp = pd.read_csv('../../data/train_fp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]
SR = settings['SR']

skf = StratifiedKFold(n_splits=settings['n_splits'], shuffle=True, random_state=42)

train_tp["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp["species_id"])):
    train_tp.iloc[val_index, -1] = fold_id

In [12]:
device = torch.device("cuda:0")
warnings.simplefilter("ignore")

for use_fold in range(5):
    print(f'---------- fold {use_fold} ----------')
    train_file_list = train_tp.query("fold != @use_fold")
    val_file_list = train_tp.query("fold == @use_fold")

    print("[fold {}] train: {}, val: {}".format(use_fold, len(train_file_list), len(val_file_list)))
    
    # loaders
    loaders = {
        "train": DataLoader(
            RfcxDataSet(train_file_list, **settings['train_params']),
            shuffle=True,
            **settings['dataloder_params']
        ),
        "valid": DataLoader(
            RfcxDataSet(val_file_list, **settings['val_params']),
            shuffle=False,
            **settings['dataloder_params']
        )
    }

    # model
    model = EfficientNetSED(**settings['model_params'])
    model.to(device)

    # Optimizer
    optimizer = Adam(model.parameters(), **settings['optim_params'])

    # Scheduler
    scheduler = wu.SimpleCosineAnnealingWarmup(optimizer, **settings['scheduler_params'])

    # Loss
    criterion = ImprovedFocalLoss(settings['loss_params']).to(device)

    # callbacks
    callbacks = [
        LWLRAP(),
        CheckpointCallback(save_n_best=1)
    ]

    runner = SupervisedRunner(
        device=device,
        input_key="waveform",
        input_target_key="targets")

    runner.train(
        model=model,
        criterion=criterion,
        loaders=loaders,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=settings['epochs'],
        verbose=True,
        logdir=f"{settings['log_path']}/{settings['test_name']}/fold{use_fold}",
        callbacks=callbacks,
        main_metric="LWLRAP",
        minimize_metric=False
    )
     
    del model, loaders, optimizer, runner
    gc.collect()

---------- fold 0 ----------
[fold 0] train: 972, val: 244
Loaded pretrained weights for efficientnet-b0
1/35 * Epoch (train): 100% 61/61 [00:28<00:00,  2.13it/s, LWLRAP=0.214, loss=2.094]
1/35 * Epoch (valid): 100% 16/16 [00:05<00:00,  3.09it/s, LWLRAP=0.300, loss=3.557]
[2021-01-24 12:58:07,233] 
1/35 * Epoch 1 (_base): lr=0.0006 | momentum=0.9000
1/35 * Epoch 1 (train): LWLRAP=0.1867 | epoch_LWLRAP=0.2137 | loss=4.2835
1/35 * Epoch 1 (valid): LWLRAP=0.1869 | epoch_LWLRAP=0.2998 | loss=3.4557
2/35 * Epoch (train): 100% 61/61 [00:29<00:00,  2.09it/s, LWLRAP=0.282, loss=1.945]
2/35 * Epoch (valid): 100% 16/16 [00:06<00:00,  2.29it/s, LWLRAP=0.140, loss=3.303]
[2021-01-24 12:58:43,843] 
2/35 * Epoch 2 (_base): lr=0.0007 | momentum=0.9000
2/35 * Epoch 2 (train): LWLRAP=0.2160 | epoch_LWLRAP=0.2821 | loss=2.1258
2/35 * Epoch 2 (valid): LWLRAP=0.2248 | epoch_LWLRAP=0.1396 | loss=3.0784
3/35 * Epoch (train): 100% 61/61 [00:30<00:00,  2.03it/s, LWLRAP=0.333, loss=1.741]
3/35 * Epoch (valid):

KeyboardInterrupt: 

In [ ]:
def prediction_for_clip(audio_id: str,
                        clip: np.ndarray, 
                        model: EfficientNetSED,
                        threshold=0.5):
    PERIOD = 10
    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)

        start = end - (5 * SR)
        end += 5 * SR
        
        #mel = create_mel(y_batch)
        audios.append(y_batch)
        
        if len_y < end:
            break
            
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    array = np.asarray(audios)
    image = torch.from_numpy(array).to(device)
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    

    with torch.no_grad():
        prediction = model(image)
        frame_pred = torch.sum(
            torch.sigmoid(torch.max(prediction["framewise_output"], 1)[0]), 0
        ).detach().cpu().numpy()
        framewise_outputs = torch.max(prediction["framewise_output"], 0)[0].detach(
            ).cpu().numpy()
        
    thresholded = framewise_outputs >= threshold
    
    for target_idx in range(thresholded.shape[1]):
        if thresholded[:, target_idx].mean() == 0:
            pass
        else:
            detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
            head_idx = 0
            tail_idx = 0
            while True:
                if (tail_idx + 1 == len(detected)) or (
                        detected[tail_idx + 1] - 
                        detected[tail_idx] != 1):
                    onset = 0.01 * detected[
                        head_idx] + global_time
                    offset = 0.01 * detected[
                        tail_idx] + global_time
                    onset_idx = detected[head_idx]
                    offset_idx = detected[tail_idx]
                    max_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].max()
                    mean_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].mean()
                    estimated_event = {
                        "audio_id": audio_id,
                        "ebird_code": target_idx,
                        "onset": onset,
                        "offset": offset,
                        "max_confidence": max_confidence,
                        "mean_confidence": mean_confidence
                    }
                    estimated_event_list.append(estimated_event)
                    head_idx = tail_idx + 1
                    tail_idx = tail_idx + 1
                    if head_idx >= len(detected):
                        break
                else:
                    tail_idx += 1
        global_time += PERIOD
        
    prediction_df = pd.DataFrame(estimated_event_list)
    return prediction_df, frame_pred

In [ ]:
def prediction(test_df: pd.DataFrame,
               model: dict,
               threshold=0.5):
    #model = get_model(model_config, weights_path)
    unique_audio_id = test_df.recording_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    frame_dict = dict()
    for audio_id in tqdm(unique_audio_id):
        clip, _ = librosa.load(
            f'/home/yuigahama/kaggle/rfcx/data/test/{audio_id}.flac',
            sr=SR,
            mono=True,
            res_type="kaiser_fast"
        )
        
        test_df_for_audio_id = test_df.query(
            f"recording_id == '{audio_id}'").reset_index(drop=True)
        prediction_df, frame_pred = prediction_for_clip(
            audio_id,
            clip=clip,
            model=model,
            threshold=threshold
        )
        frame_dict[audio_id] = frame_pred
        prediction_dfs.append(prediction_df)
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df, frame_dict

In [ ]:
device = torch.device("cuda:0")

preds = []
frams = [] 
for use_fold in range(5):

    model =  EfficientNetSED(**settings['model_params']).to(device)
    params = torch.load(f"{settings['log_path']}/{settings['test_name']}/fold{use_fold}/checkpoints/best.pth")
    model.load_state_dict(params["model_state_dict"])
    
    prediction_df, frame_dict = prediction(
        test_df=submission,
        model=model,
        threshold=0.5
    )
    print(len(prediction_df.audio_id.unique()))
    preds.append(prediction_df)
    frams.append(pd.DataFrame(frame_dict).T)

In [ ]:
sub = pd.DataFrame(np.zeros((len(submission), 24)),columns=pred_target, index=submission['recording_id'])
for p,j in zip(frams, [0.2, 0.2, 0.2, 0.2, 0.2]):
    p.columns = pred_target
    sub += (p * j)
#sub /= 5
sub.reset_index().to_csv(f"{settings['log_path']}/{settings['test_name']}/submission_{settings['test_name']}.csv", index=False)
sub